In [1]:
# 📁 notebooks/1_data_cleaning.ipynb

import pandas as pd

# Load data
df = pd.read_excel('../data/BaseMedicale_Talend.xlsx', sheet_name='Patients')

# Print column names for debugging
print("Columns in Excel file:", df.columns.tolist())

# Define expected columns to drop
columns_to_drop = ['cin', 'nom', 'prenom', 'date_admission', 'date_sortie']

# Drop unnecessary columns if they exist
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

# Create duree_sejour if date columns exist
if 'date_admission' in df.columns and 'date_sortie' in df.columns:
    df['duree_sejour'] = (pd.to_datetime(df['date_sortie']) - pd.to_datetime(df['date_admission'])).dt.days
else:
    print("⚠️ Warning: 'date_admission' or 'date_sortie' not found. Setting 'duree_sejour' to 0.")
    df['duree_sejour'] = 0

# Convert numIntervention to numeric if it exists
if 'numIntervention' in df.columns:
    df['numIntervention'] = df['numIntervention'].apply(lambda x: int(x.replace('int-', '')) if isinstance(x, str) else x)
else:
    print("⚠️ Warning: 'numIntervention' not found. Setting to 0.")
    df['numIntervention'] = 0

# Create TrancheAge if age exists
if 'age' in df.columns:
    bins = [0, 18, 40, 60, 80, 120]
    labels = ["0-17", "18-39", "40-59", "60-79", "80+"]
    df['TrancheAge'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)
else:
    print("⚠️ Warning: 'age' not found. Setting 'TrancheAge' to 'inconnu'.")
    df['TrancheAge'] = 'inconnu'

# Define expected columns (excluding mortalite, which is the target)
expected_columns = ['age', 'sexe', 'type_sanguin', 'maladie', 'id_service', 
                   'medecin_traitant', 'personnel', 'id_medicament', 
                   'readmission', 'numIntervention', 'duree_sejour', 'TrancheAge', 'mortalite']

# Add missing columns with default values
for col in expected_columns:
    if col not in df.columns:
        if col in ['age', 'readmission', 'numIntervention', 'duree_sejour']:
            df[col] = 0
        elif col == 'mortalite':
            df[col] = 'non'  # Default value for target
        else:
            df[col] = 'inconnu'

# Keep only expected columns
df = df[expected_columns]

# Save cleaned data
df.to_csv('../data/cleaned_data.csv', index=False)

print("✅ Cleaned data saved to ../data/cleaned_data.csv")
print("Columns in cleaned data:", df.columns.tolist())
df.head()

Columns in Excel file: ['cin', 'nom', 'prenom', 'mortalite', 'age', 'sexe', 'type_sanguin', 'maladie', 'date_admission', 'date_sortie', 'readmission', 'medecin_traitant', 'id_service', 'numIntervention', 'personnel', 'id_medicament']
⚠️ Warning: 'date_admission' or 'date_sortie' not found. Setting 'duree_sejour' to 0.
✅ Cleaned data saved to ../data/cleaned_data.csv
Columns in cleaned data: ['age', 'sexe', 'type_sanguin', 'maladie', 'id_service', 'medecin_traitant', 'personnel', 'id_medicament', 'readmission', 'numIntervention', 'duree_sejour', 'TrancheAge', 'mortalite']


,age,sexe,type_sanguin,maladie,id_service,medecin_traitant,personnel,id_medicament,readmission,numIntervention,duree_sejour,TrancheAge,mortalite
0,56,F,B-,Maladie liée à Pédiatre,ser-100,med-3615,pers-4637,MED-507,2,2020,0,40-59,non
1,13,F,B+,Maladie liée à Pédiatre,ser-101,med-4814,pers-8303,MED-606,10,9498,0,0-17,oui
2,56,M,O+,Maladie liée à Gastro-entérologue,ser-102,med-5012,pers-7408,MED-445,9,6325,0,40-59,non
3,69,F,A-,Maladie liée à Généraliste,ser-103,med-9830,pers-9199,MED-972,10,9609,0,60-79,oui
4,45,F,O+,Maladie liée à Pédiatre,ser-104,med-3803,pers-6651,MED-147,2,6140,0,40-59,non
